In [1]:
import os
import sys
import nltk
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.testset import TestsetGenerator
import pandas as pd
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
sys.path.insert(1, '/home/jovyan/work/code')

# injected from Developer Secrets configured in Aspire AppHost
openAiKey = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = openAiKey

path = "docs-aspire-eval/"
num_qa_to_generate=50
openai_model='gpt-4o'

loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

generator_llm = LangchainLLMWrapper(ChatOpenAI(model=openai_model, temperature=0.1))  
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset = generator.generate_with_langchain_docs(docs, testset_size=num_qa_to_generate)



Applying HeadlinesExtractor:   0%|          | 0/46 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/51 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/56 [00:00<?, ?it/s]

Property 'summary' already exists in node '2d2eb2'. Skipping!
Property 'summary' already exists in node '7d6df8'. Skipping!
Property 'summary' already exists in node '2fe860'. Skipping!
Property 'summary' already exists in node 'f4c61d'. Skipping!
Property 'summary' already exists in node '3c1b4f'. Skipping!
Property 'summary' already exists in node '64d7e7'. Skipping!
Property 'summary' already exists in node 'bab3ef'. Skipping!
Property 'summary' already exists in node 'e24c1a'. Skipping!
Property 'summary' already exists in node 'ff630c'. Skipping!
Property 'summary' already exists in node '72564c'. Skipping!


...

Property 'summary_embedding' already exists in node 'f4c61d'. Skipping!
Property 'summary_embedding' already exists in node '2d2eb2'. Skipping!
Property 'summary_embedding' already exists in node '2fe860'. Skipping!
Property 'summary_embedding' already exists in node '7d6df8'. Skipping!
Property 'summary_embedding' already exists in node '64d7e7'. Skipping!
Property 'summary_embedding' already exists in node '72564c'. Skipping!
Property 'summary_embedding' already exists in node '3c1b4f'. Skipping!
Property 'summary_embedding' already exists in node 'bab3ef'. Skipping!
Property 'summary_embedding' already exists in node 'e24c1a'. Skipping!
Property 'summary_embedding' already exists in node 'ff630c'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/51 [00:00<?, ?it/s]

In [1]:
dataset.to_pandas()
type(dataset)

file_name= f"{openai_model}_ReducedAspireDocs_{num_qa_to_generate}"

df = dataset.to_pandas()
df.to_pickle(f"{file_name}.pkl")
df.to_csv(f"{file_name}.csv")

df1 =  pd.read_pickle(f"{file_name}.pkl")
df1


NameError: name 'dataset' is not defined

In [5]:
# Run our pipeline and ingest 
